In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd

c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

In [37]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16,input_dim=13, kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(32,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(32,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(8,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam,
                loss='binary_crossentropy',
                metrics=['acc'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 16)                224       
_________________________________________________________________
dense_21 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_22 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 9         
Total params: 2,097
Trainable params: 2,097
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
checkpoint_filepath = './best_model/ckpt{epoch}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_filepath)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
model.load_weights(latest)

./best_model\ckpt_52


In [42]:
# preprocess the data (don't put this block in build app)
dataset = pd.read_csv('heart.csv', sep=';')
data = dataset[~dataset.isin(['?'])]
data = data.dropna(axis=0)
# Ubah target 2,3,4 jadi 1
data.loc[data['target'] > 1, 'target'] = 1
data = data.apply(pd.to_numeric)
X = np.array(data.drop(['target'], 1))
y = np.array(data['target'])

# Re-evaluate the model unutk semua dataset
loss,acc = model.evaluate(X,  y, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

297/297 - 0s - loss: 0.3746 - acc: 0.8586
Restored model, accuracy: 85.86%


In [43]:
predictY = model.predict(X)
hasilPredict = (predictY >= 0.5).astype(int)
from sklearn.metrics import classification_report,roc_auc_score,roc_curve
print(classification_report(y,hasilPredict))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       160
           1       0.87      0.82      0.84       137

    accuracy                           0.86       297
   macro avg       0.86      0.86      0.86       297
weighted avg       0.86      0.86      0.86       297

